# 見積もり・データは一設計関連

In [80]:
# %pip install pandas
# %pip install altair
# %pip install tablate

---

## 論物変換

TODO:

## Buffer Layout

TODO:

## DMA Scheme

TODO:

---

## 誤り訂正

悩んだので ECC 内臓の上位モデルを参考に設計する

- w/o ECC (こちらを使用している) [KIOXIA_TC58NVG0S3HTA00](https://mm.digikey.com/Volume0/opasdata/d220001/medias/docus/506/KIOXIA_TC58NVG0S3HTA00_Rev2.00_E191001C.pdf)
- w/ ECC [KIOXIA_TC58BVG0S3HTA00](https://mm.digikey.com/Volume0/opasdata/d220001/medias/docus/506/KIOXIA_TC58BVG0S3HTA00_REV2.00_E20191001C.pdf)

### Datasheet確認

#### ECC & Sector definition for ECC

- 2KByte pageは 512byte * 4area + 16byte * 4area で使用している
- main + spare の 512+16=528byteごとに、 9bitの検出と8bitの訂正能力を備える
- ECC parity code は 2112-2175 (64byte) に格納するので、ここにはアクセスできないという注意書き

main+spareごとECCのペアになっている点、ECC Status Readの戻り値がArea事に分かれていること、最後のECC parity code格納から以下のように見える

| sector | main [byte] | spare [byte] | ECC parity [byte] |
| ------ | ----------- | ------------ | ----------------- |
| 1st    | 0-511       | 2048 - 2063  | 2112 - 2127       |
| 2nd    | 512 - 1023  | 2064 - 2079  | 2128 - 2143       |
| 3rd    | 1024 - 1535 | 2080 - 2095  | 2144 - 2159       |
| 4th    | 1536 - 2047 | 2096 - 2111  | 2160 - 2175       |

data = 528byte (4224bit), parity = 16byte (128bit) で同じ訂正を組むことにする...と思ったが、 `9bitの検出と8bitの訂正能力を備える` と言うのが気になる。
一般(?)にいうECCは、1bitの訂正と2bitの検出が限界だったように思う。
ECCではない独自方式なのか、もしくは最大8bit検出というのはmain/spareを更に8分割したうえで、それぞれ独立した領域でエラーが出たときの理論的な条件でのCorrectableなのか。

以下を読むとCypress社のNOR Flashでは32byteごとにECC Encodeを施していて、1bit訂正2bit検出を拡張ハミング符号で実装している文言がある。
(総括部分の保証が若干謎の言い回しな気もするが...) 大きな単位ではなく細かめに保護をかけてもよいのかもしれない。

[SECDEC ECCスキームが2ビットを超えるエラーを処理する方法 - KBA222012 - Community Translated (JA) - infineaon Developer Community](https://community.infineon.com/t5/%E3%83%8A%E3%83%AC%E3%83%83%E3%82%B8%E3%83%99%E3%83%BC%E3%82%B9%E3%82%A2%E3%83%BC%E3%83%86%E3%82%A3%E3%82%AF%E3%83%AB-KBA/SECDEC-ECC%E3%82%B9%E3%82%AD%E3%83%BC%E3%83%A0%E3%81%8C2%E3%83%93%E3%83%83%E3%83%88%E3%82%92%E8%B6%85%E3%81%88%E3%82%8B%E3%82%A8%E3%83%A9%E3%83%BC%E3%82%92%E5%87%A6%E7%90%86%E3%81%99%E3%82%8B%E6%96%B9%E6%B3%95-KBA222012-Community-Translated-JA/ta-p/254390)

(17) Reliability Guidance によると 8bit ECC for each 512byte 必要 (これは w/o ECC modelにも記載あり) とある。
512byte中にどの程度誤りが発生するかによるが、2bit化けると訂正不能というのはギリギリを攻めているので、更に細かい単位で保護してあげようということだろうか。
ECCの重ねがけ (32byte保護した範囲を更に大きな単位で保護する? かもしくはその逆で大きな範囲で保護したものを細分化する？) みたいなことも考えたが、ECCメモリ関連の情報をざっと調べる限りは見当たらないので今回はやめておく。

ついでに同じ商で見つけたが0 data paddingをやめろという記載もあり。このUSBメモリにはAll Zeroのバイナリを置かないでください、とは言えないので対策が必要だろう...。
場当たり的な対策も考えはしたが、高速シリアル伝送などでもDC balance取るようなエンコーディングは使われていたのでこれをもとに考えていた。
元データに対してテーブル置換で実装できる8b10b encodeとその符号化率を上げたものを今回は検討にいれることにした。実用性としてこれでいいのかは正直良くわからないが、All Zero dataにはならなそうなので良いとする。

気になる点が処理順である。ECC -> 8b/10b だと、復号化するときに化けたものをもとに8b/10b decodeして最初のエラーよりも大きな誤りが伝搬してしまうように考える。
そのため、元データ -> 8b/10bしたものを書き込む元データとして捉え、これをECC Encodeするのが良さそうだ。

生成したECC Parity codeの置き場所も気になる。先述の表だと2112-2175byteに束ねられているように推察できる。
仮に32byte単位でECC Encodeを施しているとしたら元データの直後に配置することも可能だが (e.g. data0, parity0, data1, parity1, ... )
おそらくユーザーが書いたところはオリジナルのままにしたいとか、都度Encode/Decode挟んだりデータシートにあるPartial Programのことを考えると無理だったのだろう。
今回どちらにするか悩んだが、データ反転が起きなければそのままデータが使えたほうがいいと思うので、これはw/ ECC modelを見習うことにする
対象のデータはDC balance encode済なので、きっとECC Parity codeもAll Zeroにはならない想定（多分）。ECC Parity codeをまとめた部分自体に対しての訂正は不要と考える
（オリジナルデータ+Parity部分のセットなので、オリジナルから計算した値からparity code自体のエラーも検出・訂正できると思われる）

以下Codeで見積もりした結果出力

|    |   dc_balance_encode_size.data_bits |   dc_balance_encode_size.total_bits |   ecc_encode_size.data_bits |   ecc_encode_size.parity_bits |   code_rate |   surplus_bytes |   max_ecc_error_detection_count |
|---:|-----------------------------------:|------------------------------------:|----------------------------:|------------------------------:|------------:|----------------:|--------------------------------:|
| 21 |                                 64 |                                  66 |                         502 |                             9 |    0.950758 |              22 |                               8 |
| 48 |                                128 |                                 130 |                         247 |                             8 |    0.95     |              21 |                              16 |
| 49 |                                128 |                                 130 |                         502 |                             9 |    0.965385 |              55 |                               8 |

`surplus_bytes` が格納データ+Encode冗長部を全体から差し引いた空きbyte。デバッグ用データなどを入れ込めそう
`max_ecc_error_detection_count` がECC Errorで検出可能な最大数（ただし全範囲独立して起きた場合）。若干見づらいので書き直したものが以下。

128b/130b EncodeのパフォーマンスやCode Sizeに問題がなければ、64b/66b Encodeは棄却して良さそうに見える。
ECCの保護単位を256byte,512byte (w/ parity) どちらで構成するかは、少ない単位で構成できていたほうが体制が上がりそう。一方で空きbyte数の余裕観点で管理データがどうしても21byteで収められないなら512byteで構成するしかない。


| | ECC                 | DC Balance | 自由に使える空きbyte | pageあたりのECC保護分割数    | 採択時のpros |
|-|---------------------|-----------:|--------------------:|----------------------------:|-------------|
|1|拡張Hamming(256, 247)| 128b/130b  | 21                  | 8.5                         |特別な理由がなければ採用。推奨ECC厚生よりも更に細分化して構成できているので、エラー多発時の訂正能力が高い|
|2|拡張Hamming(512, 502)| 128b/130b  | 55                  | 4.25                        |USB Hostからの書き込み以外の管理データを配置するケースで、これが21byte/22byte以上必要な場合|
|3|拡張Hamming(512, 502)| 64b/66b    | 22                  | 4.25                        |128b/130b encodeの実装・動作上の問題がある場合|

#### 空きbyteの使い方

どう使うかはあまり考えていなかったが（実装中に欲しくなったときに使う事になりそう）、先の採択prosについて考えていてECCの保護単位を細かくすることのデメリットを考えていなかった。
ずばり検査数が倍になるのである。ECC分割数分だけ検査が必要なのでParity計算と一致を確認しなければならない。

RP2040ではあまり余裕のないプロセッサだと思うので、2k dataの読み出しのたびにこの計算をやるのはちょっと辛い気がする。
そもそも訂正が必要かどうかに気づくことが高速に行えればよいので、CRCなどを付与するのは良いかと考えた。
幸いRP2040は (おそらくUSB IPで使用する目的だと思うが) DMAのSnoopでCRC計算ができるので、これを付与するのが良さそうに思う。

これまたEncode同様適用順序に悩むが、CRCはあくまで訂正の要否判断なので、最終的なデータでCRC検査できることが望ましい。
またDMAでSnoopする都合上、Data bufferの一番最初か最後に付与しておいて、それ以外全部の領域をチェインなし一度のDMA Kickで済ませられるとよさそう。
DMAのDataUnitSizeを4byteにすることも鑑みると、4byte単位で開けておく。

ECC Parity codeを末尾にまとめる話は書いていたが、一方で128b/130bなどのDC Balanceは余剰bit合わせてのbalancingなので離すことは望まれない。これを踏まえ、以下の配置を検討した

##### 拡張Hamming(256, 247) + 128b/130b

| description             | area [byte] | ecc: 拡張Hamming(256, 247) |
| ----------------------  | ----------- | -------------------------- |
| usb host data0          |    0 -  511 | 512byte data |
| dc balance extra0       |  512 -  519 | 128b/130b encode additional `512[byte]*(130[bit]/128[bit])-512[byte]=8[byte]`|
| usb host data1          |  520 - 1031 | 512byte data |
| dc balance extra1       | 1032 - 1039 | 128b/130b encode additional `512[byte]*(130[bit]/128[bit])-512[byte]=8[byte]`|
| usb host data2          | 1040 - 1551 | 512byte data |
| dc balance extra2       | 1552 - 1559 | 128b/130b encode additional `512[byte]*(130[bit]/128[bit])-512[byte]=8[byte]`|
| usb host data3          | 1560 - 2071 | 512byte data |
| dc balance extra2       | 2072 - 2079 | 128b/130b encode additional `512[byte]*(130[bit]/128[bit])-512[byte]=8[byte]`|
| reserved (for last ecc) | 2080 - 2091 | ECC保護単位の最後端数分。`align_u32(2080[byte]*8[bit/byte]/247[data bit/ecc unit]/8[bit/byte])=12[byte]` 分のあまり。|
| ecc parity code         | 2092 - 2171 | `2092[byte]*8[bit/byte]/247[data bit/ecc unit]=68[ecc unit], align_u32(68[ecc unit]*9[bit/ecc unit]/8[bit/byte])=80[byte]` |
| crc code                | 2172 - 2175 | 4byte crc (codeに4byte使わないなら0埋め) |

Data, DC Balance, ECCに属さないデータ部分は `reserved (for last ecc) 12byte` +  `CRC code 4byte` = 16byte。

先のEncode時の空きbyte見積もりでは自由に使えるbyteは最低でも21byteなので、reservedに詰めるデータが5byte以内ならいずれも採択可能。

別案も計算してみたが、 拡張Hamming(256, 247) + 128b/130b がかなりきれいな値になっているのでこれを採用したい。


##### 拡張Hamming(512, 502) + 128b/130b

ECC保護対象のデータ配置は差分なし。Hammingの符号長が伸びたことでECC Protection Unitが大きくなる分Parity code sizeは減少。

後述の方式でreservedを切り詰める方法を使ったので、ここでは切り詰めではなく `no protection area` として明示した。no_protection_area内だけで不足分を0埋めして保護しても良さそう。

no_protection_areaも12byteあるが、ECC最終保護Unitがの空き部分が40byteもあるのがデバッグ向けにはありがたいかもしれない。

| description             | area [byte] | ecc: 拡張Hamming(256, 247) |
| ----------------------  | ----------- | -------------------------- |
| usb host data0          |    0 -  511 | 512byte data |
| dc balance extra0       |  512 -  519 | 128b/130b encode additional `512[byte]*(130[bit]/128[bit])-512[byte]=8[byte]`|
| usb host data1          |  520 - 1031 | 512byte data |
| dc balance extra1       | 1032 - 1039 | 128b/130b encode additional `512[byte]*(130[bit]/128[bit])-512[byte]=8[byte]`|
| usb host data2          | 1040 - 1551 | 512byte data |
| dc balance extra2       | 1552 - 1559 | 128b/130b encode additional `512[byte]*(130[bit]/128[bit])-512[byte]=8[byte]`|
| usb host data3          | 1560 - 2071 | 512byte data |
| dc balance extra2       | 2072 - 2079 | 128b/130b encode additional `512[byte]*(130[bit]/128[bit])-512[byte]=8[byte]`|
| reserved (for last ecc) | 2080 - 2119 | ECC保護単位の最後端数分 `40byte` |
| ecc parity code         | 2120 - 2159 | `44[byte]` |
| no protected area       | 2160 - 2171 | 保護領域の端数 `12[byte]` この範囲で狭く作ってもいいかも? |
| crc code                | 2172 - 2175 | 4byte crc (codeに4byte使わないなら0埋め) |

##### 拡張Hamming(512, 502) + 64b/66b

64b/66bに変更したことでDC Balance用のデータサイズが8->16byteに増加。この時点で8byte*4ent=32byte分余裕がない。

ECC Protection Unitが大きくなった分Parity code sizeは減少。

| description             | area [byte] | ecc: 拡張Hamming(256, 247) |
| ----------------------  | ----------- | -------------------------- |
| usb host data0          |    0 -  511 | 512byte data |
| dc balance extra0       |  512 -  527 | `16[byte]`|
| usb host data1          |  528 - 1039 | 512byte data |
| dc balance extra1       | 1040 - 1055 | `16[byte]`|
| usb host data2          | 1056 - 1567 | 512byte data |
| dc balance extra2       | 1568 - 1583 | `16[byte]`|
| usb host data3          | 1584 - 2095 | 512byte data |
| dc balance extra2       | 2096 - 2111 | `16[byte]`|
| reserved (for last ecc) | 2112 - 2125 | ECC保護単位の最後端数分 `40byte` が本来のアラインサイズだが、足りなくなるので後詰した残り `14byte` に切り詰め（最終ECC DataBitの後半は0埋めするしかなさそう） |
| ecc parity code         | 2126 - 2171 | `44[byte]`  |
| crc code                | 2172 - 2175 | 4byte crc (codeに4byte使わないなら0埋め) |


### 共通実装

In [160]:
import math
from dataclasses import asdict, dataclass, field

import pandas as pd


@dataclass(frozen=True)
class DcBalanceEncodeSize:
    """
    DC Balance Encodeの設定
    """

    data_bits: int
    total_bits: int

    @property
    def code_rate(self) -> float:
        return self.data_bits / self.total_bits


@dataclass(frozen=True)
class EccEncodeSize:
    """
    ECCの設定
    """

    parity_bits: int
    additional_bits: int
    data_bits: int

    @property
    def total_bits(self) -> int:
        return self.data_bits + self.parity_bits + self.additional_bits

    @property
    def code_rate(self) -> float:
        return self.data_bits / self.total_bits

    @classmethod
    def from_parity_bits(
        cls, parity_bits: int, additional_bits: int = 1
    ) -> "EccEncodeSize":
        """
        Create an ECC frame size from the number of parity bits and additional bits.
        :param parity_bits: The number of parity bits.
        :param additional_bits: The number of additional bits.
        :return: The ECC frame size.
        """
        data_bits = 2**parity_bits - parity_bits - 1
        return cls(parity_bits, additional_bits, data_bits)


@dataclass(frozen=False)
class EncodeConfig:
    """
    NAND ICのPage不揮発化におけるエンコード設定
    """

    ###############################################################
    # ユーザー設定

    # DC Balance Encodeの設定
    dc_balance_encode_size: DcBalanceEncodeSize

    # ECCの設定
    ecc_encode_size: EccEncodeSize

    ###############################################################
    # ユーザー設定可能だが、基本変更する必要はない

    # USB Hostからのデータ最小単位
    logical_data_bytes: int = 512

    # 1ページあたりにUSB Hostからのデータを何個格納するか
    logical_data_num_per_page: int = 4

    # 1pageあたりのデータ量
    bytes_per_page = 2048 + 128

    # ECC保護すべきデータサイズの最小単位
    require_ecc_protection_bytes: int = 512

    ###############################################################
    # __post_init__で計算する値

    # 1pageあたりのUSB Hostから書き込まれるデータサイズ
    logical_data_bytes_per_page: int = field(default=None)

    # 1pageあたりの冗長データ量
    redundant_bytes: int = field(default=None)

    # 符号化率
    code_rate: float = field(default=None)

    # logical_data_bytes_per_pageに対して、冗長データ量を割り当てたときの、合計のデータ量
    require_bytes: int = field(default=None)

    # 冗長データ量が足りるかどうか
    is_sufficient_total_bytes: int = field(default=None)

    # 冗長データで余るバイト数
    surplus_bytes: int = field(default=None)

    # 冗長データが足りないバイト数
    short_bytes: int = field(default=None)

    # ECC保護が十分かどうか
    is_sufficient_ecc_protection: int = field(default=None)

    # ECC保護で require_ecc_protection_bytes で指定された中で検出できるエラーの最大数. 保護範囲が複数に分かれる場合はそれぞれで発生する理想ケースを想定
    max_ecc_error_detection_count: int = field(default=None)

    def __post_init__(self):
        self.logical_data_bytes_per_page = (
            self.logical_data_bytes * self.logical_data_num_per_page
        )
        self.redundant_bytes = self.bytes_per_page - self.logical_data_bytes_per_page
        self.code_rate = (
            self.ecc_encode_size.code_rate * self.dc_balance_encode_size.code_rate
        )
        self.require_bytes = math.floor(
            self.logical_data_bytes_per_page * (1.0 / self.code_rate)
        )
        self.is_sufficient_total_bytes = self.require_bytes <= self.bytes_per_page
        self.surplus_bytes = (
            self.bytes_per_page - self.require_bytes
            if self.is_sufficient_total_bytes
            else 0
        )
        self.short_bytes = (
            self.require_bytes - self.bytes_per_page
            if not self.is_sufficient_total_bytes
            else 0
        )
        self.is_sufficient_ecc_protection = (
            self.ecc_encode_size.total_bits <= self.require_ecc_protection_bytes
        )
        self.max_ecc_error_detection_count = math.floor(
            self.require_ecc_protection_bytes * 8 / self.ecc_encode_size.total_bits
        )


### Data Layout設計時のreserved見積もり

In [172]:
def data_align(data: int, align_bytes: int) -> int:
    """Align data to the next multiple of align_bytes."""
    return math.ceil(data/align_bytes)*align_bytes


calc_config_list = [
    EncodeConfig(dc_balance_encode_size=DcBalanceEncodeSize(128, 130), ecc_encode_size=EccEncodeSize.from_parity_bits(8)),
    EncodeConfig(dc_balance_encode_size=DcBalanceEncodeSize(128, 130), ecc_encode_size=EccEncodeSize.from_parity_bits(9)),
    EncodeConfig(dc_balance_encode_size=DcBalanceEncodeSize(64, 66), ecc_encode_size=EccEncodeSize.from_parity_bits(9)),
]

for c in calc_config_list:
    # ECC保護対象全データ
    ecc_protection_bytes = c.logical_data_bytes * c.logical_data_num_per_page
    ecc_protection_bits = ecc_protection_bytes * 8

    data_encode_additional_bytes = math.ceil(c.logical_data_bytes * (c.dc_balance_encode_size.total_bits / c.dc_balance_encode_size.data_bits))-c.logical_data_bytes

    ecc_data_bits = c.ecc_encode_size.data_bits
    ecc_parity_bits = c.ecc_encode_size.parity_bits + c.ecc_encode_size.additional_bits

    ecc_protection_entry_n = math.ceil(ecc_protection_bits / ecc_data_bits)
    ecc_protection_total_parity_bits = ecc_protection_entry_n * ecc_parity_bits
    ecc_protection_total_parity_bytes = data_align(ecc_protection_total_parity_bits / 8, align_bytes=4) 

    print(c)
    print(f"data_encode_additional_bytes={data_encode_additional_bytes}")
    print(f"ecc_protection_entry_n={ecc_protection_entry_n}")

    print(f"ecc_protection_entry_n={ecc_protection_entry_n}")
    print(f"ecc_protection_total_parity_bytes={ecc_protection_total_parity_bytes}")

    reserved_bits = ecc_protection_bits % ecc_data_bits
    reserved_bytes = data_align(reserved_bits / 8, align_bytes=4)

    print(f"reserved_bytes={reserved_bytes}")
    print("")

EncodeConfig(dc_balance_encode_size=DcBalanceEncodeSize(data_bits=128, total_bits=130), ecc_encode_size=EccEncodeSize(parity_bits=8, additional_bits=1, data_bits=247), logical_data_bytes=512, logical_data_num_per_page=4, require_ecc_protection_bytes=512, logical_data_bytes_per_page=2048, redundant_bytes=128, code_rate=0.9500000000000001, require_bytes=2155, is_sufficient_total_bytes=True, surplus_bytes=21, short_bytes=0, is_sufficient_ecc_protection=True, max_ecc_error_detection_count=16)
data_encode_additional_bytes=8
ecc_protection_entry_n=67
ecc_protection_entry_n=67
ecc_protection_total_parity_bytes=76
reserved_bytes=12

EncodeConfig(dc_balance_encode_size=DcBalanceEncodeSize(data_bits=128, total_bits=130), ecc_encode_size=EccEncodeSize(parity_bits=9, additional_bits=1, data_bits=502), logical_data_bytes=512, logical_data_num_per_page=4, require_ecc_protection_bytes=512, logical_data_bytes_per_page=2048, redundant_bytes=128, code_rate=0.9653846153846154, require_bytes=2121, is_suff

### DC Balance Encode + ECC Encode時の効率・余剰byte数計算

In [161]:
import altair as alt
from tabulate import tabulate

pd.set_option("display.max_columns", 256)
pd.set_option("display.max_rows", 256)

# 見積もり
dc_balance_encode_size_list = [
    DcBalanceEncodeSize(8, 10),
    DcBalanceEncodeSize(64, 66),
    DcBalanceEncodeSize(64, 67),
    DcBalanceEncodeSize(128, 130),
    # DcBalanceEncodeSize(1, 1),  # disable
]

parity_len_n = 16  # 計算対象にするparity長
ecc_encode_size_list = [
    EccEncodeSize.from_parity_bits(i) for i in range(2, parity_len_n)
]

encode_config_list = [
    EncodeConfig(dc_balance_encode_size=dc, ecc_encode_size=ecc)
    for dc in dc_balance_encode_size_list
    for ecc in ecc_encode_size_list
]
encode_config_df = pd.json_normalize(([asdict(c) for c in encode_config_list]))


def print_chart(df: pd.DataFrame):
    base_chart_1 = (
        alt.Chart(df)
        .mark_point()
        .encode(
            alt.X("ecc_encode_size\\.data_bits:N"),
            alt.Row("dc_balance_encode_size\\.data_bits:N"),
        )
        .properties(width=300, height=100)
    )
    base_chart_2 = (
        alt.Chart(df)
        .mark_rect()
        .encode(
            alt.X("ecc_encode_size\\.data_bits:N"),
            alt.Y("dc_balance_encode_size\\.data_bits:N"),
        )
        .properties(width=300, height=300)
    )
    alt.vconcat(
        # 横軸: ECCのデータビット数, 縦軸: 冗長データ量, 行: DC Balance Encodeのデータビット数
        alt.hconcat(
            base_chart_1.encode(
                alt.Y("surplus_bytes:Q"),
                alt.Color("is_sufficient_total_bytes:N"),
            ),
            base_chart_1.encode(
                alt.Y("code_rate:Q"),
                alt.Color("is_sufficient_ecc_protection:N"),
            ),
        ).resolve_scale(color="independent"),
        # Encode設定と余裕度のヒートマップ
        alt.hconcat(
            base_chart_2.encode(
                alt.Color("surplus_bytes:Q"),
            ),
            base_chart_2.encode(
                alt.Color("code_rate:Q"),
            ),
        ).resolve_scale(color="independent"),
    ).resolve_scale(color="independent").display()


# 今回の要件を満たせるもの
available_encode_config_df = encode_config_df.query(
    "is_sufficient_total_bytes and is_sufficient_ecc_protection"
)
print(
    tabulate(
        available_encode_config_df[
            [
                "dc_balance_encode_size.data_bits",
                "dc_balance_encode_size.total_bits",
                "ecc_encode_size.data_bits",
                "ecc_encode_size.parity_bits",
                # "ecc_encode_size.additional_bits",
                "code_rate",
                "surplus_bytes",
                "max_ecc_error_detection_count",
            ]
        ],
        headers="keys",
        tablefmt="pipe",
    )
)

# 全体
print_chart(encode_config_df)
print_chart(available_encode_config_df)
available_encode_config_df


|    |   dc_balance_encode_size.data_bits |   dc_balance_encode_size.total_bits |   ecc_encode_size.data_bits |   ecc_encode_size.parity_bits |   code_rate |   surplus_bytes |   max_ecc_error_detection_count |
|---:|-----------------------------------:|------------------------------------:|----------------------------:|------------------------------:|------------:|----------------:|--------------------------------:|
| 21 |                                 64 |                                  66 |                         502 |                             9 |    0.950758 |              22 |                               8 |
| 48 |                                128 |                                 130 |                         247 |                             8 |    0.95     |              21 |                              16 |
| 49 |                                128 |                                 130 |                         502 |                             9 |    0.965385 |   

alt.VConcatChart(...)

alt.VConcatChart(...)

,logical_data_bytes,logical_data_num_per_page,require_ecc_protection_bytes,logical_data_bytes_per_page,redundant_bytes,code_rate,require_bytes,is_sufficient_total_bytes,surplus_bytes,short_bytes,is_sufficient_ecc_protection,max_ecc_error_detection_count,dc_balance_encode_size.data_bits,dc_balance_encode_size.total_bits,ecc_encode_size.parity_bits,ecc_encode_size.additional_bits,ecc_encode_size.data_bits
21,512,4,512,2048,128,0.950758,2154,True,22,0,True,8,64,66,9,1,502
48,512,4,512,2048,128,0.950000,2155,True,21,0,True,16,128,130,8,1,247
49,512,4,512,2048,128,0.965385,2121,True,55,0,True,8,128,130,9,1,502
